# Magnitude-intrinsic water-fat ambiguity can be resolved with multipeak fat modeling and a multipoint search method.

In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from TPTBox import NII, to_nii

sys.path.append("..")
sys.path.append("../../..")

from recon_mevibe import ti_ms_default

## TODO Update file paths
# Load files

# Input
path = Path("/media/data/NAKO/dataset-nako/rawdata/100/100000/mevibe/")
s_0 = path / "sub-100000_sequ-53_acq-ax_part-eco0-opp1_mevibe.nii.gz"
s_1 = path / "sub-100000_sequ-54_acq-ax_part-eco1-pip1_mevibe.nii.gz"
s_2 = path / "sub-100000_sequ-55_acq-ax_part-eco2-opp2_mevibe.nii.gz"
s_3 = path / "sub-100000_sequ-56_acq-ax_part-eco3-in1_mevibe.nii.gz"
s_4 = path / "sub-100000_sequ-57_acq-ax_part-eco4-pop1_mevibe.nii.gz"
s_5 = path / "sub-100000_sequ-58_acq-ax_part-eco5-arb1_mevibe.nii.gz"
# Target output
use_target = True
water_file = path / "sub-100000_sequ-59_acq-ax_part-water_mevibe.nii.gz"
fat_file = path / "sub-100000_sequ-63_acq-ax_part-fat_mevibe.nii.gz"
r2s_file = path / "sub-100000_sequ-61_acq-ax_part-r2s_mevibe.nii.gz" # Note in our example we have to multiply with 0.1 to get the actual value

pre = None


In [2]:

#path = Path("/media/data/NAKO/dataset-nako/rawdata/101/101000/mevibe/")
## Input
#s_0 = path / "sub-101000_sequ-53_acq-ax_part-eco0-opp1_mevibe.nii.gz"
#s_1 = path / "sub-101000_sequ-54_acq-ax_part-eco1-pip1_mevibe.nii.gz"
#s_2 = path / "sub-101000_sequ-55_acq-ax_part-eco2-opp2_mevibe.nii.gz"
#s_3 = path / "sub-101000_sequ-56_acq-ax_part-eco3-in1_mevibe.nii.gz"
#s_4 = path / "sub-101000_sequ-57_acq-ax_part-eco4-pop1_mevibe.nii.gz"
#s_5 = path / "sub-101000_sequ-58_acq-ax_part-eco5-arb1_mevibe.nii.gz"
## Target output
#use_target = True
#water_file = path / "sub-101000_sequ-59_acq-ax_part-water_mevibe.nii.gz"
#fat_file = path / "sub-101000_sequ-60_acq-ax_part-fat_mevibe.nii.gz"
#r2s_file = path / "sub-101000_sequ-62_acq-ax_part-r2s_mevibe.nii.gz" # Note in our example we have to multiply with 0.1 to get the actual value

MRI parameter:

- MagneticFieldStrength of the MRI device
- time_inversion in ms in the the same order as the $s_i$. See Dicom-Header.

In [ ]:
ti_ms = ti_ms_default
MagneticFieldStrength=3.0
print(f"{ti_ms_default= } \n{MagneticFieldStrength= }")

In [4]:

# NOTE we select slice 30 to make the example faster.

magnitude_nii:list[NII] = [to_nii(s_0),to_nii(s_1),to_nii(s_2),to_nii(s_3),to_nii(s_4),to_nii(s_5)]
magnitude = [a.get_array()for a in magnitude_nii]
if use_target:
    water = to_nii(water_file)
    fat = to_nii(fat_file)
    r2s = to_nii(r2s_file) * 0.1

In [5]:
folder = Path("test_fat_peak_model")
folder.mkdir(exist_ok=True)
# Ren marrow  #5.78, 4.33, 3.82, 3.60, 3.10, 2.8, 2.4,0.49, 0.59
# Hernando [0.50, 2.38, 2.78, 3.58, 4.24, 5.79]
models={
    #"Ren marrow":     (np.array([-3.8, -3.4, -3.1, -2.68, -2.46, -1.95, -0.5, 0.49, 0.59]), np.array([0.08991009, 0.58341658, 0.05994006, 0.08491508, 0.05994006, 0.01498501, 0.03996004, 0.00999001, 0.05694306])),
    "Hamilton liver":( np.array([-3.8, -3.4, -3.1, -2.68, -2.46, -1.95, -0.5, 0.49, 0.59]), np.array([0.088, 0.642, 0.058, 0.062, 0.058, 0.006, 0.039, 0.01 , 0.037])),
    #"UKBB1": (np.array([5.20, 4.21, 2.66, 2.00, 1.20, 0.80]), np.array([0.048, 0.039, 0.004, 0.128, 0.694, 0.087])),
    #"UKBB2": (np.array([5.20, 4.21, 2.66, 2.00, 1.20, 0.80]), np.array([0.047, 0.039, 0.006, 0.12, 0.7, 0.088])),
    #"Hernando": (np.array([-3.9, -3.5, -2.7, -2.04, -0.49, 0.50]), np.array([0.087, 0.694, 0.128, 0.004, 0.039, 0.048])),
    #"Graf 1": (np.array([-3.8, -3.4, -3.1, -2.68, -2.46, -1.95, -0.5, 0.49, 0.59]),np.array([0.11764568180678654,     0.34850433163306244 ,    0.00000001429670620434,   0.12146143325770939  ,   0.10233814607814957  ,   0.2382095897092301,   0.06436118874742854,     0.00000002971640587104,  0.18383531993907087]))
}

In [ ]:
from pipeline import predict_signal_prior
if pre is None:
    pre = predict_signal_prior(magnitude_nii,None,gpu=0,ddevice="cuda")

In [ ]:
from recon_mevibe import multipeak_fat_model_from_guess
from mago_sp import mago_sp

for name,(freqs_ppm,alpha_p) in models.items():
    for rician in [False,True]:#False
        name2 = name + "_r" if rician else name
        out_folder = folder / name2
        if (out_folder/"r2s.nii.gz").exists():
            continue
        out_w, out_f, out_r, _ = mago_sp(magnitude,water.get_array(),fat.get_array(),MagneticFieldStrength=MagneticFieldStrength,ti_ms=ti_ms,alpha_p=alpha_p,freqs_ppm=freqs_ppm,use_rician=rician)
        out_folder.mkdir(exist_ok=True)
        nii = magnitude_nii[0]
        W = nii.set_array(out_w)
        W.save(out_folder/"water.nii.gz")
        F=nii.set_array(out_f)
        F.save(out_folder/"fat.nii.gz")
        nii.set_array(out_r).save(out_folder/"r2s.nii.gz")
        dif = (W - water)
        dif2 = (F - fat)
        print(f"{name2:10} {'water':10} {dif.mean():>10.3f} {np.std(dif):>10.3f} {'fat':10} {dif2.mean():>10.3f} {np.std(dif2):>10.3f}")


In [ ]:
for name in models.keys():
    for rician in [False,True]:
        name2 = name + "_r" if rician else name
        out_folder = folder / name2
        W = NII.load(out_folder/"water.nii.gz",False)
        W.set_dtype_()
        F = NII.load(out_folder/"fat.nii.gz",False)
        F.set_dtype_()
        pdff = (W+F)
        pdff[pdff == 0] = 1
        pdff = F/pdff
        pdff_org = (water+fat)
        pdff_org[pdff_org == 0] = 1
        pdff_org = fat/pdff_org
        dif =  np.abs(W - water)
        dif2 = np.abs(F - fat)
        dif3 = np.abs(pdff_org - pdff)
        wh = np.array(pdff != 0,dtype=bool)
        print(f"{name2:16} {'water'} {dif.mean(where=wh):>10.3f} {np.std(dif,where=wh):>10.3f} {'fat'} {dif2.mean(where=wh):>10.3f} {np.std(dif2,where=wh):>10.3f} {'pdff'} {dif3.mean(where=wh):>10.3f} {np.std(dif3,where=wh):>10.3f}")


In [ ]:
print(water.max())
print(W.max())
print((W-water).max())


In [11]:
#Hamilton liver   water     10.703     18.478 fat     13.019     20.553 pdff      0.088      0.193
#Hamilton liver_r water     21.260     44.978 fat     26.502     53.153 pdff      0.048      0.101
